In [1]:
import sys
sys.path.append('..')
import os
from matplotlib import pyplot as plt
from IPython.display import display, Math, Latex

In [2]:
import gdspy
import numpy as np
from importlib import reload
from copy import deepcopy
from QCreator import elements
from QCreator import general_sample_creator as creator
from QCreator import meshing
print(meshing.__file__)

..\QCreator\meshing.py


In [3]:
import numpy as np
from scipy.constants import hbar,h,e,c
from scipy.constants import elementary_charge as elc
F0=2.0678e-15

In [4]:
e_eff=(11.45+1)/2## for all calculations
Mhz=(2*np.pi*1e6)## divide on it to obtain value in mhz
j=0.5e-3      # critical current density nA/nm^2

In [5]:
def I(f,Ec):
    Ej=Ejf(f,Ec)
    I=Ej*2*np.pi/F0*h*1e18
    return I
def Ejf(f,Ec):
    Ej=(f+Ec)**2/(8*Ec)
    return Ej
def Eji(I):#nA
    return I*F0/(2*np.pi)/1e9/h/1e9
def IEj(Ej):
    return Ej/(F0/(2*np.pi)/1e9/h/1e9)
def f(Ej,Ec):
    return np.sqrt(8*Ej*Ec)-Ec
def Ec(C):#fF
    return elc**2/(2*C)/h/1e9*1e15

In [7]:
name = 'dummy'#'PP_5_4.5'
reload(gdspy)
reload(creator)
reload(elements.pp_transmon)
reload(elements)
layers_configuration={
    'total':0,
    'restricted area':10,
    'for removing':100,
    'JJs':1,
    'flux lines':6,
    'air bridges':2,
    'air bridge pads':3,
    'vertical gridlines':15,
    'horizontal gridlines':16,
    'inverted':999
}



Couplers=[elements.pp_transmon.PP_Transmon_Coupler(0,0,20,6,'left',coupler_type = 'coupler',heightl = 0.3)
          #,elements.pp_transmon.PP_Transmon_Coupler(0,0,80,6,'right',coupler_type ='coupler',heightr = 0.5)
          #,elements.pp_transmon.PP_Transmon_Coupler(200,300,60,6,'top',coupler_type ='coupler')
          #,elements.pp_transmon.PP_Transmon_Coupler(100,600,60,6,'bottom',coupler_type ='coupler')
         ]




sample = creator.Sample(name,layers_configuration)

width = 290
height= 550
gap   = 50
g_w   = 900
g_h   = 900
g_t   = 10
b_g   = 19 # from JJ Design for JJ4q 
b_g   = 10 # for JJ_manhatten
b_w   = 16 # JJ design

a1    = 0.15 #Junction height in um
a2    = 0.30 # Junction width in um

#jj_pp = { 'a1':a1,"a2":a2,'angle_JJ':np.pi/2}
jj_pp = { 'a1':a1,"a2":a2,'angle_JJ':0,'manhatten':True,'h_w':5 ,'h_d':8 }

PP_Transmon = elements.pp_transmon.PP_Transmon(name='PP_Transmon',center=(1750,1750),
                          width = width,
                          height = height,
                          bridge_gap = b_g,
                          bridge_w   = b_w ,                   
                          gap = gap,
                          g_w = g_w,
                          g_h = g_h,
                          g_t = g_t,
                          jj_params= jj_pp,
                          layer_configuration = sample.layer_configuration,
                          Couplers = Couplers,
                          calculate_capacitance = False,
                          transformations = {}
                          )
sample.add(PP_Transmon)


sample.draw_design()

{'qubit': None, 'coupler1': Type: cpw, Position: (1258.0, 1752.5), Orientation: 0
w: 10, s: 6, g: 10
reversed: True}


In [8]:
sample.watch()

In [8]:
sample.write_to_gds()

Gds file has been writen here:  \\badwwmi\user\home\di82riv\Documents\Desktop\Master\CircuitDesign\QCreator\dummy.gds


In [9]:
Caps = sample.calculate_qubit_capacitance(cell= sample.qubit_cap_cell,mesh_volume=2)
print(Caps)

Gds file has been writen here:  \\badwwmi\user\home\di82riv\Documents\Desktop\Master\CircuitDesign\QCreator\dummy.gds
heyo
[8, 9, 10, 0]
For 0 conductor total length is:  124061
For 1 conductor total length is:  124040
For 2 conductor total length is:  4521
For 3 conductor total length is:  30640
Sum : 283262
Data has been written into the file: \\badwwmi\user\home\di82riv\Documents\Desktop\Master\CircuitDesign\QCreator\mesh_4k_data
Time for fastcap job is:  43.3580002784729
Capacitance results have been writen here:  \\badwwmi\user\home\di82riv\Documents\Desktop\Master\CircuitDesign\QCreator\mesh_4k_results
Capacitance value in:  femtofarads
           0          1         2          3
0  150.89400  -51.77332  -4.41415  -47.56522
1  -51.77332  149.40000  -0.61901  -48.21262
2   -4.41415   -0.61901  56.74710  -46.96140
3  -47.56522  -48.21262 -46.96140  252.36150


In [14]:
#Calculate Qubit frequency:
JJ_area = a1*a2*1e6 #in nm^2
print('Critical current density is: ',j*1e3, 'uA/um^2')
Ic = JJ_area*j #in nA
print('Critical current: ',Ic ,' nA')
print('Ej: ',Eji(Ic) ,' GHz')

print('junction area  in um: ',JJ_area*1e-6)


#Node 1 is Resonatorcoupler
#Node 2 is left transmon pad
#Node 3 is right transmon pad
#Node G is Ground
C12 = -Caps[0][2]#*1e3
C2G = -Caps[0][3]#*1e3
C13 = -Caps[1][2]#*1e3
C3G = -Caps[1][3]#*1e3
C23 = -Caps[0][1]#*1e3


C_sum = (4*C23+C12+C2G+C3G+C13)/4

display(Math(str('C_{sum} = '+str(C_sum)+'\\text{  fF}')))
display(Math(str('E_c = '+str( Ec(C_sum))+'\\text{  GHz}')))
display(Math(str('\\frac{\omega_q}{2\pi} ='+str(f(Eji(Ic),Ec(C_sum)))+'\\text{  GHz}')))

Critical current density is:  0.5 uA/um^2
Critical current:  22.5  nA
Ej:  11.175196062211896  GHz
junction area  in um:  0.045


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [15]:
# difference between Qcircuit and paper (the error is on the order of a few fF, in the case where the ground Capacities are
# comparable and large, the equations give the same answer) 
print(C_sum)
print(C23+(C13+C3G)*(C12+C2G)/(C13+C3G+C2G+C12))

76.97607
76.95149860623444


In [16]:
#calculate capacitive coupling
wr =    6.9  # GHz
Zr =    50    # Ohm
Rk = 25.8*1e3 # Ohm

g = wr*(C12+C13)/C_sum *(Eji(Ic)/(2*Ec(C_sum)))**0.25 * np.sqrt(np.pi*Zr/Rk)*1e3
print('C_c: ',C12+C13, ' fF')
print('g  :',g,' MHz')

C_c:  5.03316  fF
g  : 76.41800756344124  MHz


In [19]:
wq = f(Eji(Ic),Ec(C_sum))
Q = 6000
k = wr/Q


T=(1/k)*((wq-wr)/g*1e3)**2*1e-9

print(T*1e6,'us')

863.8117557332532 us
